In [55]:
import pandas as pd

In [56]:
import csv

# file_path = 'data/rum_generated.csv'
df = pd.read_csv("../data/pulsefit_rum_data.csv")
df.head(10)



,session_id,user_id,country,user_action,readable_timestamp
0,9900,33733,Nigeria,Visited Plans,25-05-25 05:17:31 AM
1,9900,33733,Nigeria,Visited Trainers,25-05-30 07:17:05 AM
2,9900,33733,Nigeria,Viewed Plan: Basic,25-08-03 11:56:23 PM
3,9900,33733,Nigeria,Visited Contact,25-10-15 12:22:17 PM
4,6735,33733,Nigeria,Viewed Plan: Pro,25-05-16 08:54:21 AM
5,6735,33733,Nigeria,Visited Plans,25-07-01 09:39:58 PM
6,6735,33733,Nigeria,Visited Trainers,25-08-27 08:32:31 PM
7,6735,33733,Nigeria,Viewed Trainer: Liam Smith,25-10-07 12:18:05 AM
8,8205,33733,Nigeria,Visited Home,25-10-21 05:54:57 AM
9,8205,33733,Nigeria,Viewed Trainer: Alex Johnson,25-09-13 08:56:44 AM


In [57]:
#Inspect df
df.info()
df.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   session_id          10000 non-null  int64 
 1   user_id             10000 non-null  int64 
 2   country             10000 non-null  object
 3   user_action         10000 non-null  object
 4   readable_timestamp  10000 non-null  object
dtypes: int64(2), object(3)
memory usage: 390.8+ KB


(10000, 5)

In [58]:
#Checking for missing data or Duplicates

#missing data
print(df.isnull().sum())

#duplicates
print(f'Duplicates: {df.duplicated().sum()}')

session_id            0
user_id               0
country               0
user_action           0
readable_timestamp    0
dtype: int64
Duplicates: 0


In [59]:
print(df['country'].value_counts())
print(df['user_action'].value_counts())


country
UK              1896
South Africa    1657
USA             1647
Nigeria         1613
Egypt           1596
Ghana           1591
Name: count, dtype: int64
user_action
Visited Trainers                1037
Visited Home                    1015
Viewed Plan: Elite              1009
Viewed Plan: Basic              1009
Viewed Trainer: Mia Carter      1007
Viewed Trainer: Alex Johnson    1003
Viewed Plan: Pro                 995
Viewed Trainer: Liam Smith       979
Visited Plans                    976
Visited Contact                  970
Name: count, dtype: int64


In [60]:
# 1) Make sure column is string and do quick cleaning
df['readable_timestamp'] = df['readable_timestamp'].astype(str).str.strip()
# remove trailing dots or accidental characters like '.' at end
df['readable_timestamp'] = df['readable_timestamp'].str.replace(
    r'\.+$', '', regex=True)
# remove duplicate spaces
df['readable_timestamp'] = df['readable_timestamp'].str.replace(
    r'\s+', ' ', regex=True)

# 2) Attempt parsing with a list of likely formats
formats_to_try = [
    "%y-%m-%d %I:%M:%S %p",   # e.g. 25-05-25 05:17:31 AM
    "%Y-%m-%d %I:%M:%S %p",   # e.g. 2025-05-25 05:17:31 AM
    "%m/%d/%Y, %I:%M %p",     # e.g. 10/29/2025, 09:45 AM
    "%m/%d/%y, %I:%M %p",     # e.g. 10/29/25, 09:45 AM
    "%y-%m-%d %H:%M:%S",      # fallback 24-hour without AM/PM
    "%Y-%m-%d %H:%M:%S"
]

# initialize a column of NaT datetimes
df['readable_dt'] = pd.NaT

for fmt in formats_to_try:
    mask = df['readable_dt'].isna()
    try:
        parsed = pd.to_datetime(
            df.loc[mask, 'readable_timestamp'], format=fmt, errors='coerce')
        df.loc[mask, 'readable_dt'] = parsed
    except Exception:
        # if a format causes unexpected error, skip it
        pass

# 3) Final fallback: let pandas infer format for remaining NaT rows
mask = df['readable_dt'].isna()
if mask.any():
    df.loc[mask, 'readable_dt'] = pd.to_datetime(
        df.loc[mask, 'readable_timestamp'], errors='coerce', infer_datetime_format=True)

# 4) Inspect how many failed to parse
failed = df['readable_dt'].isna().sum()
print(f"Parsing failures (still NaT): {failed} out of {len(df)}")

# Optionally show some problematic examples (up to 10)
if failed:
    print(df.loc[df['readable_dt'].isna(),
          'readable_timestamp'].head(10).to_list())

# 5) Create separate columns once parsed
# If you want date as ISO (YYYY-MM-DD) for Power BI, use dt.date or dt.strftime
df['date_only'] = df['readable_dt'].dt.date
df['time_only'] = df['readable_dt'].dt.strftime('%I:%M:%S %p')  # keeps AM/PM
df['hour'] = df['readable_dt'].dt.hour
df['day'] = df['readable_dt'].dt.day_name()
df['month'] = df['readable_dt'].dt.month_name()

Parsing failures (still NaT): 0 out of 10000


In [61]:
# Export cleaned data
cleaned_path = '../data/pulsefit_rum_data_cleaned.csv'
df.to_csv(cleaned_path, index=False)
print('Cleaned data saved:', cleaned_path)

Cleaned data saved: ../data/pulsefit_rum_data_cleaned.csv
